## Preparacion del dataset

Del estudio realizado en el [Data-analisis](data-analisis.ipynb)  se obtiene que los parametros importantes para modelar son: 'Species', 'Light_ISF', 'Soil', 'Sterile', 'Conspecific', 'AMF', 'EMF', 'Phenolics', 'Lignin', 'NSC' y 'Event', siendo este ultimo el target.

In [1]:
import pandas as pd

dataset = pd.read_csv("../dataset/Tree_Data.csv")

In [2]:
dataset_modif = dataset[['Species', 'Light_ISF', 'Soil', 'Sterile', 
                         'Conspecific', 'AMF', 'EMF', 'Phenolics', 'Lignin', 'NSC', 'Event']]

dataset_modif.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2783 entries, 0 to 2782
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Species      2783 non-null   object 
 1   Light_ISF    2783 non-null   float64
 2   Soil         2783 non-null   object 
 3   Sterile      2783 non-null   object 
 4   Conspecific  2783 non-null   object 
 5   AMF          2783 non-null   float64
 6   EMF          1283 non-null   float64
 7   Phenolics    2783 non-null   float64
 8   Lignin       2783 non-null   float64
 9   NSC          2783 non-null   float64
 10  Event        2782 non-null   float64
dtypes: float64(7), object(4)
memory usage: 239.3+ KB


Un solo Event es null, lo elimino y los nan del EMF los paso a 0

In [3]:
dataset_modif = dataset.dropna(subset=["Event"])
dataset_modif.fillna(value={"EMF": 0}, inplace=True)

dataset_modif

C:\Users\5p4c3\AppData\Local\Temp\ipykernel_4528\3383934828.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_modif.fillna(value={"EMF": 0}, inplace=True)


,No,Plot,Subplot,Species,Light_ISF,Light_Cat,Core,Soil,Adult,Sterile,...,AMF,EMF,Phenolics,Lignin,NSC,Census,Time,Event,Harvest,Alive
0,126,1,C,Acer saccharum,0.106,Med,2017,Prunus serotina,I,Non-Sterile,...,22.00,0.00,-0.56,13.86,12.15,4,14.0,1.0,NaN,NaN
1,11,1,C,Quercus alba,0.106,Med,2017,Quercus rubra,970,Non-Sterile,...,15.82,31.07,5.19,20.52,19.29,33,115.5,0.0,NaN,X
2,12,1,C,Quercus rubra,0.106,Med,2017,Prunus serotina,J,Non-Sterile,...,24.45,28.19,3.36,24.74,15.01,18,63.0,1.0,NaN,NaN
3,2823,7,D,Acer saccharum,0.080,Med,2016,Prunus serotina,J,Non-Sterile,...,22.23,0.00,-0.71,14.29,12.36,4,14.0,1.0,NaN,NaN
4,5679,14,A,Acer saccharum,0.060,Low,2017,Prunus serotina,689,Non-Sterile,...,21.15,0.00,-0.58,10.85,11.20,4,14.0,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2777,7086,17,C,Quercus alba,0.122,High,2017,Quercus rubra,1595,Non-Sterile,...,10.89,39.00,5.53,21.44,18.99,16,56.0,1.0,NaN,NaN
2778,7165,17,B,Prunus serotina,0.111,Med,2017,Populus grandidentata,891,Non-Sterile,...,40.89,0.00,0.83,9.15,11.88,16,56.0,1.0,NaN,NaN
2779,7217,17,D,Quercus alba,0.118,Med,2017,Acer rubrum,1468,Non-Sterile,...,15.47,32.82,4.88,19.01,23.50,16,56.0,1.0,NaN,NaN
2780,7306,17,D,Quercus alba,0.118,Med,2017,Quercus rubra,1454,Non-Sterile,...,11.96,37.67,5.51,21.13,19.10,16,56.0,1.0,NaN,NaN
